## Optimisation d'un stratifié

Ici on cherche à optimiser la rigidité d'un stratifié en fonction du nombre de plis, de leur orientation et de leur ordre. On procède en 2 approches distinctes : une première où on connaît les propriétés de chaque plis (le matériau, leur nombre, et leur orientation les uns par rapport aux autres); on peut obtenir ça par exemple par une première optimisation MINLP qu'on a vu dans d'autres cas. Dans une deuxième approche, on cherchera à faire une optimisation globale directe via un algorithme développé dans la thèse de F.X Irisarri (2009).

### 1ère approche 

#### Essais

Ici on veut d'abord tester l'optimisation d'une fonction quelconque qui ne dépend que de l'ordre de plis qui sont déjà orientés les uns par rapport aux autres. Comme pour le voyageur de commerce, on peut représenter cela par une matrice d'angles à diagonale nulle représentant les angles entre les plis numérotés de 1 à N.

##### Force brute

In [1]:
import itertools
import numpy as np
import math

# Fonction quelconque à optimiser

def calculate_cost(path, angle_matrix):
    cost = 0
    for i in range(len(path) - 1):
        cost += math.cos(angle_matrix[path[i]][path[i+1]])
    cost += math.cos(angle_matrix[path[-1]][path[0]])  
    return cost

def brute_force_tsp(angle_matrix):
    num_cities = len(angle_matrix)
    all_permutations = itertools.permutations(range(num_cities))
    
    best_path = None
    best_cost = float('inf')
    
    for path in all_permutations:
        current_cost = calculate_cost(path, angle_matrix)
        if current_cost < best_cost:
            best_cost = current_cost
            best_path = path
    
    return best_path, best_cost

# Example usage:
np.random.seed(42)  # Pour la reproductibilité

angle_matrix = np.random.rand(10, 10) * 90  
angle_matrix = (angle_matrix + angle_matrix.T)/2

best_path, best_cost = brute_force_tsp(angle_matrix)
print("Best path:", best_path)
print("Best cost:", best_cost)


Best path: (0, 8, 1, 9, 6, 5, 7, 3, 4, 2)
Best cost: -6.48303190678916


##### Recuit simulé

In [31]:
import numpy as np
import random
import math

# Fonction quelconque à optimiser

def calculate_cost(path, angle_matrix):
    cost = 0
    for i in range(len(path) - 1):
        cost += math.cos(angle_matrix[path[i]][path[i+1]])
    cost += math.cos(angle_matrix[path[-1]][path[0]])  
    return cost

# Recuit simulé

def simulated_annealing(angle_matrix, initial_temp, cooling_rate, stopping_temp):
    num_cities = len(angle_matrix)
    
    # Generate an initial solution
    current_solution = list(range(num_cities))
    random.shuffle(current_solution)
    current_cost = calculate_cost(current_solution, angle_matrix)
    
    best_solution = current_solution.copy()
    best_cost = current_cost
    
    temperature = initial_temp
    
    while temperature > stopping_temp:
        # Create a new solution by swapping two cities
        new_solution = current_solution.copy()
        i, j = random.sample(range(num_cities), 2)
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        
        new_cost = calculate_cost(new_solution, angle_matrix)
        
        # Determine if we should accept the new solution
        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
            current_solution = new_solution
            current_cost = new_cost
            
            # Update the best solution found so far
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost
        
        # Cool down the temperature
        temperature *= cooling_rate
    
    return best_solution, best_cost

initial_temp = 1000
cooling_rate = 0.99
stopping_temp = 0.00001

best_path, best_cost = simulated_annealing(angle_matrix, initial_temp, cooling_rate, stopping_temp)
print("Best path:", best_path)
print("Best cost:", best_cost)


Best path: [5, 7, 3, 4, 2, 0, 8, 1, 9, 6]
Best cost: -6.483031906789159


##### ACO

In [68]:
import numpy as np
import math
import random

# Algorithme des colonies de fourmis

class AntColony:
    def __init__(self, angle_matrix, num_ants, num_iterations, decay, alpha=1, beta=1):
        self.angle_matrix = angle_matrix
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.num_cities = len(angle_matrix)
        self.pheromone = np.ones((self.num_cities, self.num_cities)) / self.num_cities

    def run(self):
        best_path = None
        best_cost = float('inf')
        
        for i in range(self.num_iterations):
            all_paths = self.generate_all_paths()
            self.spread_pheromone(all_paths, self.num_ants)
            best_path, best_cost = self.get_best_path(all_paths, best_path, best_cost)
            print(f'------ Iteration {i} ------')
            print('current best_path :', best_path, 'current best_cost :',best_cost)
            self.pheromone * self.decay
            
        return best_path, best_cost

    def generate_all_paths(self):
        all_paths = []
        for i in range(self.num_ants):
            path = self.generate_path(0)
            all_paths.append((path, self.calculate_cost(path)))
        return all_paths

    def generate_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        
        for _ in range(self.num_cities - 1):
            move = self.pick_move(self.pheromone[prev], self.angle_matrix[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
            
        path.append((prev, start))
        return path

    def pick_move(self, pheromone, angles, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0
        
        row = pheromone ** self.alpha * ((1.0 / (angles + 1e-10)) ** self.beta)
        norm_row = row / row.sum()
        move = np_choice(range(self.num_cities), 1, p=norm_row)[0]
        return move

    def calculate_cost(self, path):
        total_cost = 0
        for move in path:
            total_cost += math.cos(self.angle_matrix[move[0]][move[1]])
        return total_cost

    def spread_pheromone(self, all_paths, num_ants):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, cost in sorted_paths[:num_ants]:
            for move in path:
                self.pheromone[move] += 1.0 / self.angle_matrix[move[0]][move[1]]

    def get_best_path(self, all_paths, best_path, best_cost):
        for path, cost in all_paths:
            if cost < best_cost:
                best_path = path
                best_cost = cost
        return best_path, best_cost

def np_choice(a, size, p):
    return np.random.choice(a, size=size, p=p)


num_ants = 1000
num_iterations = 300
decay = 0.9995
alpha = 0.3
beta = 0.0000001

ant_colony = AntColony(angle_matrix, num_ants, num_iterations, decay, alpha, beta)
best_path_aco, best_cost_aco = ant_colony.run()

print("Ant Colony Optimization - Best path:", best_path_aco, "Best cost:", best_cost_aco)


------ Iteration 0 ------
current best_path : [(0, 3), (3, 7), (7, 5), (5, 2), (2, 6), (6, 4), (4, 1), (1, 9), (9, 8), (8, 0)] current best_cost : -4.538605035020846
------ Iteration 1 ------
current best_path : [(0, 3), (3, 4), (4, 2), (2, 6), (6, 5), (5, 9), (9, 8), (8, 1), (1, 7), (7, 0)] current best_cost : -5.06328875376677
------ Iteration 2 ------
current best_path : [(0, 3), (3, 4), (4, 2), (2, 6), (6, 5), (5, 9), (9, 8), (8, 1), (1, 7), (7, 0)] current best_cost : -5.06328875376677
------ Iteration 3 ------
current best_path : [(0, 3), (3, 4), (4, 2), (2, 6), (6, 5), (5, 9), (9, 8), (8, 1), (1, 7), (7, 0)] current best_cost : -5.06328875376677
------ Iteration 4 ------
current best_path : [(0, 8), (8, 1), (1, 9), (9, 5), (5, 6), (6, 2), (2, 4), (4, 3), (3, 7), (7, 0)] current best_cost : -5.538010773976926
------ Iteration 5 ------
current best_path : [(0, 8), (8, 1), (1, 9), (9, 5), (5, 6), (6, 2), (2, 4), (4, 3), (3, 7), (7, 0)] current best_cost : -5.538010773976926
------ 

On trouve bien les mêmes résultats en paramétrant bien les algorithmes, cependant, il semble qu'un nombre important d'itérations soient nécessaires.

#### Avec salome meca

##### Sans appliquer les règles de conception des composites

In [4]:
import subprocess


# Fonctions pour calculer la rigidité via code aster
def launch_calcul():
    l1 = subprocess.check_output('wsl -d smeca /opt/sm/bin/as_run /mnt/c/Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/export')
    return(l1)

def write_in_aster(chemin):
    filename = 'C://Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/RunCase_11_Stage_1.comm'
    with open(filename, 'r') as file:
            lines = file.readlines()

    to_bewritten = f'''_F(EPAIS=5.0, 
                                   MATER={liste_mat[chemin[0]]}, 
                                   ORIENTATION={0}),'''
    for i in range(int(len(chemin))-1):
        to_bewritten += f'''\n                                _F(EPAIS=5.0, 
                                   MATER={liste_mat[chemin[i]]}, 
                                   ORIENTATION={antisymmetric_matrix[chemin[i]][chemin[i+1]]}),'''

    for k, line in enumerate(lines):
        if 'clt140 = ' in line:
            index = k
            break
    lines[index] = f'''clt140 = DEFI_COMPOSITE(identifier='5:1', 
                        COUCHE=({to_bewritten}))\n'''

    with open(filename, 'w') as file:
        file.writelines(lines)
    
    with open(filename, 'w') as file:
        file.writelines(lines)
    for i, line in enumerate(lines):
            if 'N = ' in line:
                lines[i] = f'N = {len(chemin)}\n'
                break

    with open(filename, 'w') as file:
        file.writelines(lines)

In [3]:
# Fonctions d'écriture de fichiers / lecture de résultats

def remove_section():
        file_path = 'C://Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/RunCase_11_Stage_1.comm'
        with open(file_path, 'r') as file:
            lines = file.readlines()

        start_keyword = 'clt140 = '
        end_keyword = 'fieldmat = '
        new_lines = []
        inside_section = False

        for line in lines:
            if start_keyword in line:
                new_lines.append(line)
                inside_section = True
            elif end_keyword in line and inside_section:
                inside_section = False
                new_lines.append(line)
            elif not inside_section:
                new_lines.append(line)

        with open(file_path, 'w') as file:
            file.writelines(new_lines)

def read_defo():
        file_path = 'C://Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/defo'
        defo_values = []
        start_reading = False
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if "EPXX" in line:
                    start_reading = True

                if start_reading:
                    if '=====>' in line:
                        break
                    try:
                        defo = line.split("|")
                        if len(defo) <= 5:
                            break
                        else:
                            defo = float(defo[4])
                            defo_values.append(defo)
                    except ValueError:
                        continue
        return defo_values

def read_cons():
        file_path = 'C://Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/cons'
        defo_values = []
        start_reading = False
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if "SIXX" in line:
                    start_reading = True

                if start_reading:
                    if '=====>' in line:
                        break
                    try:
                        defo = line.split("|")
                        if len(defo) <= 5:
                            break
                        else:
                            defo = float(defo[4])
                            defo_values.append(defo)
                    except ValueError:
                        continue
        return defo_values

In [71]:
# On suppose une optimisation déjà faite et on connaît les propriétés des  plis

liste_mat = ['bois1', 'bois2', 'bois1', 'bois2']
import numpy as np

# Créer une matrice 5x5 avec des éléments aléatoires entre -90 et 90
matrix = np.random.randint(-90, 91, size=(4, 4))

# Rendre la matrice antisymétrique en soustrayant sa transposee et en divisant par 2
antisymmetric_matrix = (matrix - matrix.T) / 2

# Mettre les éléments diagonaux à zéro
np.fill_diagonal(antisymmetric_matrix, 0)

##### Force brute

In [7]:
import itertools
import numpy as np
import math

def calculate_cost(path):
    remove_section()
    write_in_aster(path)
    
    launch_calcul()
    defo = max(np.abs(read_defo()))
    cons = max(np.abs(read_cons()))
    return(cons/defo)

def brute_force_tsp(angle_matrix):
    num_cities = len(angle_matrix)
    all_permutations = itertools.permutations(range(num_cities))
    
    best_path = None
    best_cost = float('inf')
    
    for path in all_permutations:
        current_cost = calculate_cost(path)
        if current_cost < best_cost:
            best_cost = current_cost
            best_path = path
        print('chemin testé :' , path, 'cout actuel :' , current_cost)
    return best_path, best_cost


best_path, best_cost = brute_force_tsp(antisymmetric_matrix)
print("Best path:", best_path)
print("Best cost:", best_cost)

chemin testé : (0, 1, 2, 3) cout actuel : 5069.585870455439
chemin testé : (0, 1, 3, 2) cout actuel : 7704.746535874708
chemin testé : (0, 2, 1, 3) cout actuel : 8769.392627620235
chemin testé : (0, 2, 3, 1) cout actuel : 9478.29209672251
chemin testé : (0, 3, 1, 2) cout actuel : 8595.303140746737
chemin testé : (0, 3, 2, 1) cout actuel : 10229.778885842972
chemin testé : (1, 0, 2, 3) cout actuel : 7308.010642715174
chemin testé : (1, 0, 3, 2) cout actuel : 5955.980180248793
chemin testé : (1, 2, 0, 3) cout actuel : 6874.9785720238215
chemin testé : (1, 2, 3, 0) cout actuel : 6518.32903088356
chemin testé : (1, 3, 0, 2) cout actuel : 10098.003374489212
chemin testé : (1, 3, 2, 0) cout actuel : 11258.74229815473
chemin testé : (2, 0, 1, 3) cout actuel : 11023.982385411435
chemin testé : (2, 0, 3, 1) cout actuel : 8751.47841401925
chemin testé : (2, 1, 0, 3) cout actuel : 9437.88467136821
chemin testé : (2, 1, 3, 0) cout actuel : 6600.118943525421
chemin testé : (2, 3, 0, 1) cout actuel 

##### Recuit simulé

On ne teste que très peu le recuit simulé car pour des nombre de plis relativement faibles il prend beaucoup d'itérations et n'est pas intéressant

In [ ]:
import random
def simulated_annealing(angle_matrix, initial_temp, cooling_rate, stopping_temp):
    num_cities = len(angle_matrix)
    
    # Generate an initial solution
    current_solution = list(range(num_cities))
    random.shuffle(current_solution)
    current_cost = calculate_cost(current_solution)
    
    best_solution = current_solution.copy()
    best_cost = current_cost
    
    temperature = initial_temp
    
    while temperature > stopping_temp:
        # Create a new solution by swapping two cities
        new_solution = current_solution.copy()
        i, j = random.sample(range(num_cities), 2)
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        
        new_cost = calculate_cost(new_solution)
        
        # Determine if we should accept the new solution
        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
            current_solution = new_solution
            current_cost = new_cost
            
            # Update the best solution found so far
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost
        
        # Cool down the temperature
        temperature *= cooling_rate
    
    return best_solution, best_cost

initial_temp = 1000
cooling_rate = 0.99
stopping_temp = 0.00001

best_path, best_cost = simulated_annealing(antisymmetric_matrix, initial_temp, cooling_rate, stopping_temp)
print("Best path:", best_path)
print("Best cost:", best_cost)

##### ACO

In [31]:
liste_mat = ['bois1', 'bois2', 'bois1', 'bois2','bois1','bois2','bois1','bois2', 'bois1', 'bois2']
import numpy as np

# Créer une matrice 5x5 avec des éléments aléatoires entre -90 et 90
matrix = np.random.randint(-90, 91, size=(4, 4))

# Rendre la matrice antisymétrique en soustrayant sa transposee et en divisant par 2
antisymmetric_matrix = (matrix - matrix.T) / 2

# Mettre les éléments diagonaux à zéro
np.fill_diagonal(antisymmetric_matrix, 0)

In [40]:
import numpy as np
import math
import itertools

# Fonction de calcul du coût (à adapter selon vos besoins)
def calculate_cost(path):
    remove_section()
    write_in_aster(path)
    
    launch_calcul()
    defo = max(np.abs(read_defo()))
    cons = max(np.abs(read_cons()))
    return -cons / defo

# Algorithme des fourmis pour le TSP
def ant_colony_tsp(angle_matrix, num_ants, num_iterations, alpha, beta, evaporation_rate):
    num_cities = len(angle_matrix)
    
    # Initialiser les phéromones
    pheromones = np.ones((num_cities, num_cities))
    best_path = None
    best_cost = float('inf')
    
    for iteration in range(num_iterations):
        all_paths = []
        all_costs = []
        
        for ant in range(num_ants):
            path = [np.random.randint(num_cities)]
            while len(path) < num_cities:
                current_city = path[-1]
                probabilities = []
                for next_city in range(num_cities):
                    if next_city not in path:
                        tau = pheromones[current_city, next_city] ** alpha
                        eta = (1 / angle_matrix[current_city, next_city]) ** beta if angle_matrix[current_city, next_city] != 0 else 1
                        probabilities.append(tau * eta)
                    else:
                        probabilities.append(0)
                
                probabilities = np.array(probabilities)
                probabilities = probabilities / probabilities.sum()
                next_city = np.random.choice(range(num_cities), p=probabilities)
                path.append(next_city)
            
            current_cost = calculate_cost(path)
            all_paths.append(path)
            all_costs.append(current_cost)
            
            if current_cost < best_cost:
                best_cost = current_cost
                best_path = path
            
            print(f'Ant {ant} path: {path}, cost: {current_cost}')
        
        # Mise à jour des phéromones
        for i in range(num_cities):
            for j in range(num_cities):
                pheromones[i, j] *= (1 - evaporation_rate)
                
        for path, cost in zip(all_paths, all_costs):
            for i in range(len(path) - 1):
                pheromones[path[i], path[i + 1]] += 1 / cost
            pheromones[path[-1], path[0]] += 1 / cost  # pour rendre le chemin circulaire
    
    return best_path, best_cost

# Paramètres
num_ants = 10
num_iterations = 10
alpha = 1.0  # Importance de la trace de phéromone
beta = 2.0   # Importance de l'heuristique
evaporation_rate = 0.5

# Création de la matrice angle_matrix
angle_matrix = antisymmetric_matrix

# Appel de l'algorithme des fourmis
best_path, best_cost = ant_colony_tsp(angle_matrix, num_ants, num_iterations, alpha, beta, evaporation_rate)
print("Best path:", best_path)
print("Best cost:", best_cost)

Ant 0 path: [2, 1, 0, 3], cost: -10469.490476478613
Ant 1 path: [1, 2, 3, 0], cost: -10670.591245268599
Ant 2 path: [0, 1, 3, 2], cost: -10594.017397646683
Ant 3 path: [0, 3, 2, 1], cost: -10670.657239190896
Ant 4 path: [2, 1, 3, 0], cost: -11493.358606013726
Ant 5 path: [3, 0, 1, 2], cost: -10468.99314911146
Ant 6 path: [3, 0, 1, 2], cost: -11512.447384235706
Ant 7 path: [3, 0, 1, 2], cost: -11314.475270567504
Ant 8 path: [0, 1, 2, 3], cost: -11372.093190648187
Ant 9 path: [3, 1, 2, 0], cost: -11512.568328733305
Ant 0 path: [1, 0, 3, 2], cost: -10757.453356353664
Ant 1 path: [3, 0, 1, 2], cost: -11374.690499230572
Ant 2 path: [1, 2, 0, 3], cost: -10426.030624240142
Ant 3 path: [3, 0, 1, 2], cost: -10468.99314911146
Ant 4 path: [0, 1, 2, 3], cost: -10508.878538523168
Ant 5 path: [1, 0, 3, 2], cost: -11478.285166706239
Ant 6 path: [3, 0, 1, 2], cost: -10468.99314911146
Ant 7 path: [3, 0, 1, 2], cost: -11374.690499230572
Ant 8 path: [0, 3, 1, 2], cost: -11040.320952094598
Ant 9 path: [3,

In [28]:
for i in range (len(best_path)):
    print(liste_mat[best_path[i]])

bois1
bois1
bois1
bois2
bois1
bois2
bois2
bois2
bois2
bois1


### 2ème approche

Dans cette approche on se réfère à la thèse citée plus haut déjà. On utilise un algorithme évolutionnaire simple en modifiant les opérateurs de mutation, de croisement... De cette manière, on peut réaliser une optimisation globale sur tous les paramètres pour maximiser la rigidité de la plaque.

In [38]:
# Redéfinir l'écriture des fichiers pour bien prendre en compte les règles de conception

def write_in_aster(chemin):
    filename = 'C://Users/TOUGERON/Documents/PRO/SMECA/Test_plaque_trou/Plate_Files/RunCase_11/Result-Stage_1/RunCase_11_Stage_1.comm'
    with open(filename, 'r') as file:
            lines = file.readlines()

    mat = []
    to_bewritten = ''
    for i in range (int(len(chemin)//2)):
        l = np.random.uniform(0,1)
        if l >=0.6:
              mat+=['bois1']
        else:
             mat+=['bois2']
    mat = mat + mat[::-1]
    for i in range(int(len(mat))):
        l = np.random.uniform(0,1)
        to_bewritten += f'''\n                                _F(EPAIS=5.0, 
                                MATER={mat[i]}, 
                                ORIENTATION={chemin[i]}),'''
         
    for k, line in enumerate(lines):
        if 'clt140 = ' in line:
            index = k
            break
    lines[index] = f'''clt140 = DEFI_COMPOSITE(identifier='5:1', 
                        COUCHE=({to_bewritten}))\n'''

    with open(filename, 'w') as file:
        file.writelines(lines)
    for i, line in enumerate(lines):
            if 'N = ' in line:
                lines[i] = f'N = {len(chemin)}\n'
                break

    with open(filename, 'w') as file:
        file.writelines(lines)

##### Sans les règles de conception

In [22]:
import random
from deap import base, creator, tools, algorithms

# Définir la fonction de calcul de rigidité
def calculate_rigidity(configuration):
    remove_section()
    write_in_aster(configuration)
    
    launch_calcul()
    defo = max(np.abs(read_defo()))
    cons = max(np.abs(read_cons()))
    res = cons / defo
    print(res)
    return res,

# Configuration GA
N_max = 30  # Nombre maximal de plis
orientations = [0, 45, 90, -45]

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def init_individual():
    n_plies = random.randint(3, N_max)  # S'assurer qu'il y a au moins 3 plis
    return creator.Individual([random.choice(orientations) for _ in range(n_plies)])

toolbox = base.Toolbox()
toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", calculate_rigidity)

def cx_custom(ind1, ind2):
    if len(ind1) > 1 and len(ind2) > 1:
        return tools.cxTwoPoint(ind1, ind2)
    else:
        return ind1, ind2

def mut_individual(individual):
    if len(individual) > 1:
        index = random.randrange(len(individual))
        individual[index] = random.choice(orientations)
    else:
        # Si l'individu n'a qu'un pli, il faut simplement changer son orientation
        individual[0] = random.choice(orientations)
    return individual,

toolbox.register("mate", cx_custom)
toolbox.register("mutate", mut_individual)
toolbox.register("select", tools.selTournament, tournsize=3)

# Paramètres GA
population = toolbox.population(n=10)
ngen = 10
cxpb = 0.5
mutpb = 0.2

# Exécuter l'algorithme génétique
result, log = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=False)

# Meilleure solution trouvée
best_individual = tools.selBest(population, k=1)[0]
print("Meilleure configuration de plis:", best_individual)
print("Rigidité maximale:", calculate_rigidity(best_individual)[0])


8773.440166329427
10373.363246309047
10019.606631851419
11611.169765929677
7341.636682403091
9641.931899383628
10742.267833620797
2122.5035863340668
6794.137051639972
10688.208816268765
11192.70778903308
11397.948782126785
11256.199283192123
10741.422182845201
10284.390023921445
11205.423507413088
10818.403384696418
10997.63969798812
7731.561656013649
11349.271777365228
10294.572535216255
11627.0905744486
10742.328624530512
10743.513155776018
10175.722207974424
11009.943524190658
11358.103192292992
11583.563478519747
11215.501047433736
11214.507542034298
11500.78165123651
11608.079175097979
10743.643896338774
10740.353630411446
11475.357116212865
10742.122466070927
9886.425334857946
11586.549107141718
11185.39444210032
11588.319783079463
11610.117069402937
11609.750500936718
10739.48894951626
11475.585032871337
11382.239050711602
11575.164501881967
10176.856922650397
11608.261512077075
10740.62643525422
11634.054597103484
11611.558653267966
11626.967330552441
11630.707795552087
11611.9

On voit clairement que les règles ne sont pas respectées, néanmoins, on arrive bien à une maximisation de la rigidité.

##### Avec les règles de conception

In [39]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Définir la fonction de calcul de rigidité
def calculate_rigidity(configuration):
    remove_section()
    write_in_aster(configuration)
    
    launch_calcul()
    defo = max(np.abs(read_defo()))
    cons = max(np.abs(read_cons()))
    res = cons / defo
    print(res)
    return res

def fitness_function(individual):
    rigidity = calculate_rigidity(individual)
    
    # Appliquer les pénalités pour les contraintes non respectées
    penalty = 0.0
    
    # Symétrie miroir
    half = len(individual) // 2
    if individual[:half] != individual[-1:-(half+1):-1]:
        penalty += 10000  # Valeur de pénalité arbitraire
    
    # Équilibrage
    if individual.count(45) != individual.count(-45):
        penalty += 10000  # Valeur de pénalité arbitraire
    if individual.count(90) != individual.count(-90):
        penalty += 10000  # Valeur de pénalité arbitraire

    # Règle des 10 %
    for orientation in orientations:
        if individual.count(orientation) < 0.1 * N_plies:
            penalty += 10000  # Valeur de pénalité arbitraire
    
    # Groupage et désorientation
    for i in range(len(individual) - 1):
        if individual[i] == individual[i + 1]:
            penalty += 10000  # Valeur de pénalité arbitraire
        if abs(individual[i] - individual[i + 1]) > 45:
            penalty += 10000  # Valeur de pénalité arbitraire
    
    return rigidity - penalty,

# Configuration GA
N_plies = 10  # Fixer le nombre de plis
orientations = [0, 45, 90, -45,-90]

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def init_individual():
    num_plies_per_orientation = N_plies // len(orientations)
    remainder = N_plies % len(orientations)
    
    base_plies = {orientation: num_plies_per_orientation for orientation in orientations}
    for i in range(remainder):
        base_plies[orientations[i]] += 1

    individual = []
    for orientation in orientations:
        individual.extend([orientation] * base_plies[orientation])

    random.shuffle(individual)
    return creator.Individual(individual)

def cx_custom(ind1, ind2):
    size = min(len(ind1), len(ind2))
    cxpoint1 = random.randint(1, size - 1)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]
    
    for i in range(len(ind1)//2):
        ind1[-(i+1)] = ind1[i]
        ind2[-(i+1)] = ind2[i]

    return ind1, ind2

def mut_individual(individual):
    size = len(individual) // 2
    index = random.randint(0, size - 1)
    new_orientation = random.choice(orientations)
    individual[index] = new_orientation
    individual[-(index + 1)] = new_orientation
    return individual,

toolbox = base.Toolbox()
toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_function)
toolbox.register("mate", cx_custom)
toolbox.register("mutate", mut_individual)
toolbox.register("select", tools.selTournament, tournsize=3)

# Paramètres GA
population = toolbox.population(n=10)
ngen = 10
cxpb = 0.5
mutpb = 0.2

# Exécuter l'algorithme génétique
for gen in range(ngen):
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    population[:] = toolbox.select(offspring + population, len(population))

# Meilleure solution trouvée
best_individual = tools.selBest(population, k=1)[0]
print("Meilleure configuration de plis qui respecte les contraintes : ", best_individual)
print("Rigidité maximale:", calculate_rigidity(best_individual))


c:\Users\TOUGERON\AppData\Local\miniconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\TOUGERON\AppData\Local\miniconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


10743.420498205962
10743.468381310038
9487.493087545883
10278.1331867115
11635.761820676422
11631.831763814344
10283.365896556452
9409.444369572806
9324.684337605848
7634.733890384327
11635.878999102779
11633.889590249963
11631.448616087091
10742.957854810202
11631.47417879235
11631.47417879235
11632.114041795516
11633.465617229893
10743.160425039876
10743.127084054566
11390.82381850759
10743.244397961182
10743.164982996359
4871.060521346704
3187.3261105286633
11282.909583595247
10743.164982996359
11633.262508294038
10743.160425039876
10743.456725690128
10743.13341567837
10743.01735013564
10743.139364516694
10743.13341567837
10743.38741971371
11636.216677303699
11633.32090248404
11633.513934861847
11633.559450986337
11633.32090248404
11633.465617229893
10743.155593669522
11633.200209662726
10743.506733036984
11633.559450986337
10743.175718261122
10743.127084054566
11633.32090248404
10743.456725690128
11633.513934861847
10743.164982996359
10743.160425039876
11633.200209662726
11633.4656

Cette fois-ci, les critères sont presque tous respectés (mise à part ici du passage de -45 à 90).